I used LightGBM as a first attempt to submit.
Not much EDA or Feature engineering, I only did encode the categoricals.
I think there are so many things for improvment, as I am just new to Kaggle.
Hope you guys like it, and please upvote it!

In this version(version 11) and previous versions,  I used parameters which has been tuned for LightGBM in another notebook.
In this version, I applied a small learning rate which is suggested by Optuna. So it takes a while to complete this notebook but I hope score changes.
Let's see how the score changes!

In version 12, I try to optimize parameters by incremental training.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestRegressor
import lightgbm as lgb
from sklearn.metrics import mean_squared_error

In [ ]:
train = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv')
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv')
sample_sub = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv')

In [ ]:
train.head()

In [ ]:
delete_columns = ['id']
train.drop(delete_columns, axis=1, inplace=True)
test.drop(delete_columns, axis=1, inplace=True)

In [ ]:
categorical_features = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6','cat7', 'cat8', 'cat9']

In [ ]:

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values)+list(test[c].values))
        train[c] = lbl.transform(train[c].values)
        test[c] = lbl.transform(test[c].values)
        
display(train.head())


In [ ]:
#delete_columns = ['cat0', 'cat1', 'cat2', 'cat3', 'cat4', 'cat5', 'cat6', 'cat7', 'cat8', 'cat9']
y_train = train['target']
X_train = train.drop('target', axis = 1)
X_test = test

In [ ]:
y_train.shape

In [ ]:
preds_list_base = []
preds_list_final_iteration = []
preds_list_all = []
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params_lgb = {
    'random_state':42,
    'metric': 'rmse',
    'n_jobs': -1,
    'cat_feature': [x for x in range(len(categorical_features))],
    'bagging_seed':42,
    'feature_fraction_seed':42,
    'learning_rate': 0.0011992715138089741,
    'max_depth': 101,
    'num_leaves': 86,
    'reg_alpha': 7.504329214783163,
    'reg_lambda': 1.5631184517427836,
    'colsample_bytree': 0.22354989226986266,
    'min_child_samples': 149,
    'subsample_freq': 4,
    'subsample': 0.5143496951794435,
    'max_bin': 720,
    'min_data_per_group': 55,
    'cat_smooth': 78,
    'cat_l2': 7,
    'n_estimators': 20000
}

y_preds = 0

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    #lgb_train = lgb.Dataset(X_tr, y_tr, categorical_feature=categorical_features)
    #lgb_eval = lgb.Dataset(X_val, y_val, reference=lgb_train, categorical_feature=categorical_features)
    
    Model = LGBMRegressor(**params_lgb).fit(X_tr, y_tr,
                                            valid_sets=[(X_val, y_val)],
                                            eval_metric =['rmse'],
                                            categorical_feature=['categorical_features'],
                                            verbose_eval=0,
                                            early_stopping_rounds=250)
    
    preds_list_base.append(Model.predict(X_test))
    preds_list_all.append(Model.predict(X_test))
    print(f'RMSE for Base model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
    first_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
    params = params_lgb.copy()
    
    for i in range(1, 8):
        if i >2:
            params['reg_lambda'] *= 0.9
            params['reg_alpha'] *= 0.9
            params['num_leaves'] += 40
    
        params['learning_rate'] = 0.003
        Model = LGBMRegressor(**params).fit(X_tr, y_tr, eval_set=[(X_val, y_val)],
                                            eval_metric=['rmse'],
                                            early_stopping_rounds=200, 
                                            categorical_feature=['categorical_features'],
                                            verbose=0,
                                            init_model=Model)
                
        preds_list_all.append(Model.predict(X_test)
        #print(f'RMSE for Incremental trial {i} model is {np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))}')
                              
   #last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
   #last_rmse = np.sqrt(mean_squared_error(y_val, Model.predict(X_val)))
   #print('',end='\n\n')
   #print(f'Improvement of : {first_rmse - last_rmse}')
   #print('-' * 100)
   preds_list_final_iteration.append(Model.predict(X_test))


In [ ]:
pd.DataFrame(oof_train).to_csv('oof_train_kfold.csv', index=False)

In [ ]:
y_preds = pd.DataFrame(y_preds)

In [ ]:
y_subs = y_preds

In [ ]:
sample_sub['target'] = y_subs
sample_sub.to_csv('submission_CV.csv', index=False)